<a href="https://colab.research.google.com/github/arjunjyothieswarb/EECE5550/blob/main/Project/Global_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install git+https://gitlab.com/neu-autonomy/gym-neu-racing.git

In [ ]:
import gymnasium
import numpy as np
import gym_neu_racing
from gymnasium import spaces
from gym_neu_racing.envs.wrappers import StateFeedbackWrapper, MappingWrapper
import matplotlib.pyplot as plt
from typing import Callable
import matplotlib.cm as cmx
import matplotlib.colors as colors
from gym_neu_racing import motion_models
from gym_neu_racing import sensor_models
from gym_neu_racing.sensor_models import Lidar2D
from gym_neu_racing.sensor_models.sensor_model import SensorModel
from typing import Iterable, Optional
import gtsam
import matplotlib.pyplot as plt
import numpy as np
from gtsam import Marginals, Point2, Point3, Pose2, Pose3, Values
from gtsam.symbol_shorthand import L, X
from matplotlib import patches
from scipy.linalg import expm

# Implementation

## Setting up the environment